## Deliverable 2. Create a Customer Travel Destinations Map.

In [28]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [29]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Castro,BR,-24.7911,-50.0119,45.50,96,1,3.87,clear sky
1,1,Zlitan,LY,32.4674,14.5687,61.59,87,52,14.34,broken clouds
2,2,Hermanus,ZA,-34.4187,19.2345,57.79,63,15,14.99,few clouds
3,3,Dikson,RU,73.5069,80.5464,18.45,89,100,8.34,overcast clouds
4,4,Dauphin,CA,51.1494,-100.0502,20.03,90,100,6.69,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
max_temp=float(input("What is the maximum temperature you would like for your trip ?:"))
min_temp=float(input("What is the minimum temperature you would like for your trip ?:"))

What is the maximum temperature you would like for your trip ?:90
What is the minimum temperature you would like for your trip ?:75


In [30]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df=city_data_df.loc[((city_data_df["Max Temp"]>=min_temp)&(city_data_df["Max Temp"]<=max_temp))]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
7,7,Rikitea,PF,-23.1203,-134.9692,78.10,69,1,12.91,clear sky
10,10,Puerto Ayora,EC,-0.7393,-90.3518,75.00,90,31,6.06,scattered clouds
19,19,Banfora,BF,10.6333,-4.7667,78.57,69,80,6.96,broken clouds
21,21,Labuhan,ID,-6.8844,112.2051,88.93,57,85,4.18,overcast clouds
28,28,Bandarbeyla,SO,9.4942,50.8122,86.83,59,52,6.31,broken clouds


In [31]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [32]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
7,7,Rikitea,PF,-23.1203,-134.9692,78.10,69,1,12.91,clear sky
10,10,Puerto Ayora,EC,-0.7393,-90.3518,75.00,90,31,6.06,scattered clouds
19,19,Banfora,BF,10.6333,-4.7667,78.57,69,80,6.96,broken clouds
21,21,Labuhan,ID,-6.8844,112.2051,88.93,57,85,4.18,overcast clouds
28,28,Bandarbeyla,SO,9.4942,50.8122,86.83,59,52,6.31,broken clouds


In [33]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Rikitea,PF,78.10,clear sky,-23.1203,-134.9692,
10,Puerto Ayora,EC,75.00,scattered clouds,-0.7393,-90.3518,
19,Banfora,BF,78.57,broken clouds,10.6333,-4.7667,
21,Labuhan,ID,88.93,overcast clouds,-6.8844,112.2051,
28,Bandarbeyla,SO,86.83,broken clouds,9.4942,50.8122,
34,Port Blair,IN,87.84,scattered clouds,11.6667,92.7500,
40,Dutse,NG,83.37,overcast clouds,11.7594,9.3392,
47,Vaini,TO,80.04,scattered clouds,-21.2000,-175.2000,
49,Kapaa,US,76.98,scattered clouds,22.0752,-159.3190,
51,Buraydah,SA,77.68,clear sky,26.3260,43.9750,


In [35]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
   
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        hotel_df.loc[index, "Hotel Name"] = np.NaN
        

In [42]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Rikitea,PF,78.10,clear sky,-23.1203,-134.9692,People ThankYou
10,Puerto Ayora,EC,75.00,scattered clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
19,Banfora,BF,78.57,broken clouds,10.6333,-4.7667,Residences David Hebié
21,Labuhan,ID,88.93,overcast clouds,-6.8844,112.2051,DoubleFAG
28,Bandarbeyla,SO,86.83,broken clouds,9.4942,50.8122,JABIR HOTEL
...,...,...,...,...,...,...,...
697,Road Town,VG,75.29,scattered clouds,18.4167,-64.6167,"ZINGARA 76ft Catamaran, full crew-all included"
698,Hwange,ZW,77.00,overcast clouds,-18.3693,26.5019,Hwange Colliery Guest House
701,Matay,EG,78.13,clear sky,28.4190,30.7792,Nag Aespen
702,Balikpapan,ID,77.99,light rain,-1.2675,116.8289,Swiss-Belhotel Balikpapan


In [43]:
# 8a. Create the output File (CSV)
output_data_file='WeatherPy_Vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [44]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [47]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]

# Call the figure to plot the data.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))